## **CA2-AI**
# **Amir-Hossein Shahgholi(810199441)**

### Part1-Genetic

In first part we want to implementing a genetic algorithm for portfolio management:

At first we should create our Portfolio class to representation our chromosomes.

In [ ]:
class Portfolio:
    data = list
    fitness_result = int
    
    def __init__(self, data):
        self.data = data

In our Chromosome class we have a list of data that represent amount of each shares in our Protfolio.

Now we should create our population first. We fill data with some random numbers that sum of them will become 1.
To do this we first generate some random numbers and then normalize the numbers.</br>
Below function will do that for us.

In [ ]:
import random

def create_sample_data(number_of_data):
    numbers = []
    for _ in range(number_of_data-1):
        random_no = random.random()
        numbers.append(random_no)
    total = sum(numbers)
    numbers = [x/total for x in numbers]
    return numbers

So we create sample portfolio with randoms data that represent our population.

In [ ]:
NUMBER_OF_CHOROMOSOME_IN_POPULATION = 1000
NUMBER_OF_SHARES = 400

population = []

for _ in range(NUMBER_OF_CHOROMOSOME_IN_POPULATION):
    sample_data = create_sample_data(NUMBER_OF_SHARES)
    temp_portfolio = Portfolio(data=sample_data)
    population.append(temp_portfolio)

Now create our population with sample random data.

In the next step we should define a fitness function to calculate a fitness value for each portfolio.
For fitness function first we find total_returns of our portfolio and then we should find risk of our portfolio using the portfolio's standard deviation and then we calculate a fitness value with this formula: **fitness = (portfolio_return - risk_penalty * portfolio_risk) / return_penalty**

In [ ]:
import numpy as np

risk_penalty = 0.6
return_penalty = 1000


def calculate_fitness(returns: list, risks: list, portfolio):

    returns = np.array(returns)
    risks = np.array(risks)

    shares_percent = np.array(portfolio.data)
    portfolio_return = np.dot(returns, shares_percent)
    portfolio_risk = np.sqrt(np.dot(shares_percent**2, risks))
    fitness = (portfolio_return - risk_penalty * portfolio_risk) / return_penalty
    return fitness

To find crossover of two chromosomes we use uniform crossover and then after finding child's data we normalize the data to have some of one in child's data. 

In [ ]:
def crossover(portfolio1, portfolio2) -> Portfolio:
    pdata1 = portfolio1.data 
    pdata2 = portfolio2.data
    child_data = []
    for i in range(NUMBER_OF_SHARES):
        if random.randint(0, 1) == 0:
            child_data.append(pdata1[i])
        else:
            child_data.append(pdata2[i])
    total = sum(child_data)
    child_data = [x/total for x in child_data]
    return Portfolio(child_data)